## Import Required Libraries :

In [1]:
from sklearn.cluster import KMeans 
#import clustering from sklearn.cluster using KMeans
from sklearn.utils import shuffle 
#import shuffle utility from sklearn utilities
import numpy as np 
#import numpy library as np
import csv 
#import csv import and export format for spreadsheets and databases
import math 
#import math library 
import matplotlib.pyplot 
#import pythons math plot library
from matplotlib import pyplot as plt 
#import plot from math plot library 
import pandas as pd 
#imports pandas methods as pd since python built-in methods can overlap panda methods.

## Pre-Processing of Human Observed Dataset : 

In [2]:
dataFrame_same_gsc = pd.read_csv('same_pairs_gsc.csv')
#print(dataFrame_same)
dataFrame_diff_gsc = pd.read_csv('diffn_pairs_gsc.csv')
#print(dataFrame_diff)
dataFrame_diff_random_gsc = dataFrame_diff_gsc.sample(n=len(dataFrame_same_gsc))
#print(dataFrame_diff_random)
same_diff_one_gsc = dataFrame_same_gsc.append(dataFrame_diff_random_gsc)
#print(same_diff_one)
same_diff_shuffle_gsc = shuffle(same_diff_one_gsc)
#print(same_diff_shuffle) 
dataFrame_features_gsc = pd.read_csv('GSC-Features.csv')
#print(dataFrame_human_features)

get_features_gsc = pd.merge(same_diff_shuffle_gsc,dataFrame_features_gsc, how='left', left_on='img_id_A', right_on='img_id')
#print(get_features_gsc)
get_features_gsc.drop(columns = ['img_id_A', 'img_id'], axis=1,inplace=True)
get_features_gsc=pd.merge(get_features_gsc,dataFrame_features_gsc,how='left',left_on='img_id_B',right_on='img_id')
get_features_gsc.drop(columns = ['img_id_B', 'img_id'], axis=1,inplace=True)

#features_sub = pd.DataFrame(columns=['f1_x', 'f2_x', 'f3_x', 'f4_x', 'f5_x', 'f6_x', 'f7_x', 'f8_x', 'f9_x'])

features_final_gsc=get_features_gsc.loc[:,:]
feature_target_gsc=get_features_gsc.loc[:,"target"]


print(features_final_gsc)
print(feature_target_gsc)








        target  f1_x  f2_x  f3_x  f4_x  f5_x  f6_x  f7_x  f8_x  f9_x   ...    \
0            1     0     0     0     0     0     0     0     0     0   ...     
1            0     1     1     0     0     0     0     0     0     0   ...     
2            0     1     1     0     0     0     0     0     0     0   ...     
3            1     0     0     0     0     0     0     0     0     0   ...     
4            0     0     0     0     0     0     0     0     0     0   ...     
5            1     0     1     0     0     0     0     0     0     0   ...     
6            1     0     1     0     0     0     0     0     0     0   ...     
7            1     0     0     1     0     0     0     0     0     0   ...     
8            1     0     0     1     0     1     0     0     0     0   ...     
9            0     0     0     0     0     0     0     0     0     0   ...     
10           0     1     0     0     0     0     0     1     0     0   ...     
11           0     0     0     0     0  

In [3]:
features_final_gsc.drop(columns = ['target'], axis=1,inplace=True)
features_final_gsc.to_csv('New_Features_gsc.csv', index=False, header=False)

In [4]:
feature_target_gsc.to_csv('New_Target_gsc.csv', index=False, header=False)

## Initialize Different Parameters & Functions : 

In [5]:
maxAcc = 0.0 
#initialize Max Accurary = 0.0 

maxIter = 0 
#initialize Max Iterations = 0

C_Lambda = 3
#initialize Lambda value

TrainingPercent = 80 
#initialize Training Data Set % = 80

ValidationPercent = 10 
#initialize Validation Data Set % = 10

TestPercent = 10 
#initialize Test Data Set % = 10 

M = 10 
#initialize number of M Basis Functions = 10

PHI = [] 
#initialize PHI as an Array 

IsSynthetic = False 
#set IsSynthetic value as False

## Calculation for Logistic Regression :

In [14]:
def GetTargetVector(filePath):                         #define GetTarget Vector code block with source filePath
    t = []                                             #define target vector array
    with open(filePath, 'rU') as f:                    #open or set directory for the source filePath as f
        reader = csv.reader(f)                         #open filePath defined as f in csv reader 
        for row in reader:                             #from the filepath access rows 
            t.append(int(row[0]))                      #append row values to target array
    return t                                           #return the values to target dataset t

def GenerateRawData(filePath, IsSynthetic): 
                                 #define GenerateRawData with source filePath and IsSynthetic value set as False    
    dataMatrix = []              #define dataMatrix as an array to store values
    with open(filePath, 'rU') as fi: #open or set directory for the source filePath as fi
        reader = csv.reader(fi)      #open filePath defined as fi in csv reader
        for row in reader:           #from the filepath access rows
            dataRow = []             #dataRow array saves the values
            for column in row: #for every column in datarow
                dataRow.append(float(column)) #append the value of dataRow as float
            dataMatrix.append(dataRow)        #append the dataRow matrix in dataMatrix   
    
    if IsSynthetic == False : 
        dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1) # delete array elements from dataMatrix 
    dataMatrix = np.transpose(dataMatrix)                       # take the transpose of the dataMatrix     
    return dataMatrix                                  #return the dataMatrix which was transposed in above line

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80): 
                                                    #define GenerateTrainingTarget data with data and percentage
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01))) 
                                                   #covert to float and put the values in TrainingLen
    t           = rawTraining[:TrainingLen]        #put the values of rawTraining set into target vector array t
                                                 
    return t                                       #return new target vector array t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80): 
                                                  #define GenerateTrainingDataMatrix with rawData and percentage
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
                                                  #covert to float and put the values in T_Len
    d2 = rawData[:,0:T_len]                       #put values in d2 
    return d2                                     #return d2 with changes

def GenerateValData(rawData, ValPercent, TrainingCount): 
                                              #define GenerateValData with rawData, TrainingCount and ValPercent 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01)) #calculate val_size with float values
    V_End = TrainingCount + valSize 
    dataMatrix = rawData[:,TrainingCount+1:V_End]             #update the dataMatrix with new values 
    return dataMatrix                                         #return the updated dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
                                                               #define GenerateValTargetVector for taget vectors 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))     #calculate val_size with float values
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]                          #update the target Vector values with new values
    return t                                                   #return the updated array t

def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic): 
                                                               #define GenerateBigSigma for traning Vectors
    BigSigma    = np.zeros((len(Data),len(Data)))              #BigSigma takes the values of the Data
    DataT       = np.transpose(Data)                           #take the transpose of the Data Matrix 
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01)) #calculate Training Lenth with float values       
    varVect     = [] 
    for i in range(0,len(DataT[0])):                           #iterate the values for tranpose data range
        vct = []
        for j in range(0,int(TrainingLen)):                    #iterate the values for TrainingLen range
            vct.append(Data[i][j])                             #append the values in array vct 
        varVect.append(np.var(vct))                            #append the values in array varVect
    
    for j in range(len(Data)):                                 #iterate the values for data range
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:                                    #if value True
        BigSigma = np.dot(3,BigSigma)                          #calculate BigSigma
    else: #if value False
        BigSigma = np.dot(200,BigSigma)                        #calculate BigSigma
        Add_Noise_BigSigma = BigSigma+0.00001*np.random.rand(1024,1024) #Add Noise to Big Sigma
    return Add_Noise_BigSigma                                           #return the updated value of the BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):                       #define the GetScalar
    R = np.subtract(DataRow,MuRow)                             #calculate R
    T = np.dot(BigSigInv,np.transpose(R))                      #calculate T   
    L = np.dot(R,T)                                            #calculate L
    return L                                                   #return the updated value of L 

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):               #define the RadialBasis function    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv)) 
    return phi_x                                               #return the update value of phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80): #define the Phi Matrix 
    DataT = np.transpose(Data)                                    #take the transpose of DataT matrix
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))    #calculate TraningLen with float values         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix)))              #take the required PHI values
    BigSigInv = np.linalg.inv(BigSigma)                           #calculate the BigSigma inverse value
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv) #calculate the PHI values
    return PHI                                                              #return updated PHI values

def GetWeightsClosedForm(PHI, T, Lambda):                                    #define the GetWeightsCLosedForm
    Lambda_I = np.identity(len(PHI[0]))                                      #Lamda_I = identity matrix of PHI
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)                                          #calculate phi transpose
    PHI_SQR     = np.dot(PHI_T,PHI)                                          #calculate phiT.phi
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)                                   #calculate lambda + phiT.phi
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)                                  #calculate phi inverse
    INTER       = np.dot(PHI_SQR_INV, PHI_T)              #calculate dot product of phi inverse and transpose
    W           = np.dot(INTER, T)                                           #calculate w
    return W                                                                 #return the updated value of w

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):  #define the GetPhiMatrix
    DataT = np.transpose(Data)                                     #transpose of DataT
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))     #calculate TraningLen with float values         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix)))               #Phi matrix  
    BigSigInv = np.linalg.inv(BigSigma)                            #calculate the BigSigma Inverse
    for  C in range(0,len(MuMatrix)): 
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv) 
                                                                      #for each range of Mu Matrix calculate phi
    return PHI                                                        #return the updated values of Phi

def GetValTest(VAL_PHI,W):                                            #define GetValTest for validation
    Y = np.dot(W,np.transpose(VAL_PHI))                               #calculate W . ValPhi
    return Y                                                          #return the update values of Y

def GetErms(VAL_TEST_OUT,ValDataAct):                                 #define the GetERMS block
    sum = 0.0                                                         #initialize sum = 0.0
    t=0                                                               #initialize t = 0 for target vectors
    accuracy = 0.0                                                    #initialize accuracy = 0.0
    counter = 0                                                       #initialize counter to 0
    val = 0.0                                                         #initialize val to 0
    for i in range (0,len(VAL_TEST_OUT)): 
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)     #calculate the sum
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1                                                #increment the counter
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))        #calculate the accuracy of the data
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT)))) # return the value of ERMS

## Fetch and Prepare Dataset :

In [15]:
RawTarget = GetTargetVector('New_Target_gsc.csv')              #feed the machine the target ouput csv file
RawData   = GenerateRawData('New_Features_gsc.csv',IsSynthetic)#feed the machine the concatenated output csv file

C:\Users\NEERAJ\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\NEERAJ\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'U' mode is deprecated
  if sys.path[0] == '':

KeyboardInterrupt



## Prepare Training Data :

In [ ]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent)) #give traning target data
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)         #give training data from matrix

print(TrainingTarget.shape) 
print(TrainingData.shape)

## Prepare Validation Data :

In [ ]:
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget)))) 
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget))) 
#give the validation data with generated data

print(ValDataAct.shape)
print(ValData.shape)

## Prepare Test Data :

In [ ]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct))) #give test data 

print(ValDataAct.shape)
print(ValData.shape)

## Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix ] :

In [ ]:
ErmsArr = []                  
#define the Erms array

AccuracyArr = []              
#define accuracy array

kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData)) 
#calculate the clustering using KMeans

Mu = kmeans.cluster_centers_  
#calculate the Mu value

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)   
#calculate the BigSigma value

TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)         
#calculate the traning phi value

W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
#calculate the regularizer value

TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
#get the Test phi value

VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100) 
#get the val phi value

In [ ]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

## Finding Erms on Training, Validation and Test Set : 

In [ ]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W) 
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget)) #get erms of training data accuracy
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))    #get erms of validation data accuracy
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))       #get erms of test data accuracy


In [ ]:
print ('UBITname      = nabhyank')
print ('Person Number = 50290958')
print ('----------------------------------------------------')
print ("------------------LeToR Data------------------------")
print ('----------------------------------------------------')
print ("-------Closed Form with Radial Basis Function-------")
print ('----------------------------------------------------')
print ("M = " + str (M))
print ("Lambda = " + str(C_Lambda))
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("Traning Accuracy   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("Validation Accuracy   = " + str(float(ValidationAccuracy.split(',')[0])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))
print ("Testing Accuracy   = " + str(float(TestAccuracy.split(',')[0])))

## Gradient Descent solution for Logistic Regression :

In [ ]:
print ('----------------------------------------------------')
print ('--------------Please Wait for 2 mins!----------------')
print ('----------------------------------------------------')

In [ ]:
W_Now        = np.dot(220, W)                                        #take the dot product to calculate W now 
La           = 4                                                     #initialize Lambda value
learningRate = 0.03                                                  #set the learning rate value
L_Erms_Val   = []                                                    #define Erms value of validation data array
L_Erms_Acc_Val   = []                                                #define Erms value of accuracy data array
L_Erms_TR    = []                                                    #define Erms value of training data array
L_Erms_Acc_TR   = []                                                 #define Erms value of accuracy data array
L_Erms_Test  = []                                                    #define Erms value of testing data array
L_Erms_Acc_Test   = []                                               #define Erms value of accuracy data array
W_Mat        = []                                                    #define W

for i in range(0,400):                                               #iterate for range of 40 
    
    a = math.exp(-np.dot(np.transpose(W_Now),TRAINING_PHI[i]))
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     = -np.dot((TrainingTarget[i] - (1/(1+a))),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,a) 
    #calculate change in E_W dot product with lambda
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W) 
    #calculate change in E    
    Delta_W       = -np.dot(learningRate,Delta_E) 
    #change in W is negative of dot product of learning rate and change in E  
    W_T_Next      = W_Now + Delta_W 
    #next W = current w + change in W
    W_Now         = W_T_Next 
    #update current W to next W where W is the weight
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(a,W_T_Next) 
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget) 
    #calculate the erms training accuracy for all weights and phi values
    L_Erms_TR.append(float(Erms_TR.split(',')[1])) 
    #append the values acquired in above calculation
    L_Erms_Acc_TR.append(float(Erms_TR.split(',')[0]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(a,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct) 
    #calculate the erms validation accuracy for all values of testing set and actual data values
    L_Erms_Val.append(float(Erms_Val.split(',')[1])) 
    #append the values acquired in above calculation
    L_Erms_Acc_Val.append(float(Erms_Val.split(',')[0]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(a,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct) 
    #calculate the erms testing accuracy 
    L_Erms_Test.append(float(Erms_Test.split(',')[1])) 
    #append the values from above calculation
    L_Erms_Acc_Test.append(float(Erms_Test.split(',')[0])) 
    #append the values from above calculation

In [ ]:
print ('----------Gradient Descent Solution--------------------')
print ("M = "+ str (M))
print ("La = "+ str(La))
print ("Learning Rate = "+ str(learningRate))
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("Traning Accuracy   = " + str(np.around(max(L_Erms_Acc_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("Validation Accuracy   = " + str(np.around(max(L_Erms_Acc_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
print ("Testing Accuracy   = " + str(np.around(max(L_Erms_Acc_TR),5)))

plt.figure(1)
plt.subplot(2,2,1)
plt.plot(L_Erms_TR)
plt.title('Training ERMS')
plt.xlabel('Number of data points given:')

plt.subplot(2,2,2)
plt.plot(L_Erms_Acc_TR)
plt.title('Training Accuracy')
plt.xlabel('Number of data points given:')

plt.figure(2)
plt.subplot(2,2,1)
plt.plot(L_Erms_Val)
plt.title('Validation ERMS')
plt.xlabel('Number of data points given:')

plt.subplot(2,2,2)
plt.plot(L_Erms_Acc_Val)
plt.title('Validation Accuracy')
plt.xlabel('Number of data points given:')

plt.figure(3)
plt.subplot(2,2,1)
plt.plot(L_Erms_Test)
plt.title('Testing ERMS')
plt.xlabel('Number of data points given:')

plt.subplot(2,2,2)
plt.plot(L_Erms_Acc_Test)
plt.title('Testing Accuracy')
plt.xlabel('Number of data points given:')

plt.show()